In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go

i=116

a = f'F:/Intern_work/converted_csv_dataset/training_csv/{i:06}.csv'
b = f'F:/Intern_work/training/labels/{i:06}.csv'
c = f'F:/Intern_work/calib/{i:06}.csv'

# Load point cloud data using pandas
ptCloud_df = pd.read_csv(a) #############################################
csv_file_name = b  # Replace with the actual CSV file name #################################
tv2c_df = pd.read_csv(c, header=None) #################################################################


############   caculate_3d.ipynb
import os
import numpy as np
import pandas as pd

# Step 1: Define the path to the folder containing the CSV labels file



folder_path = 'F:/Intern_work/training/labels'  # Update with your folder path 

# Step 2: Load the CSV labels file
csv_file_path = os.path.join(folder_path, csv_file_name)
df = pd.read_csv(csv_file_path)

# Step 3: Loop through each row to process the data and compute 3D corners
corner_data_list = []

for idx, row in df.iterrows():
    h = row['height']   # Height
    w = row['width']    # Width
    l = row['length']   # Length
    x_pos = row['x']    # X position
    y_pos = row['y']    # Y position
    z_pos = row['z']    # Z position
    rotation_y = row['rotation_y']  # Rotation angle

    # Step 3.1: Compute corners in the local object coordinate system
    # Step 1: Compute corners in the local object coordinate system
    x_corners = [l/2, l/2, l/2, -l/2, -l/2, -l/2, l/2, -l/2]
    y_corners = [0, 0, -h, 0, -h, 0, -h, -h]
    z_corners = [w/2, -w/2, w/2, w/2, w/2, -w/2, -w/2, -w/2]











    corners_3d = np.vstack([x_corners, y_corners, z_corners])

    # Step 3.2: Rotation matrix around Y-axis
    R_y = np.array([
        [np.cos(rotation_y), 0, np.sin(rotation_y)],
        [0, 1, 0],
        [-np.sin(rotation_y), 0, np.cos(rotation_y)]
    ])

    # Step 3.3: Rotate the corners to world coordinates
    corners_3d_rotated = np.dot(R_y, corners_3d)

    # Step 3.4: Translate the corners to the object's position in the world
    corners_3d_world = corners_3d_rotated + np.array([[x_pos], [y_pos], [z_pos]])

    # Step 3.5: Combine corner coordinates into a list for easier addition to DataFrame
    corner_data = []
    for i in range(8):
        corner_data.append((corners_3d_world[0, i], corners_3d_world[1, i], corners_3d_world[2, i]))

    corner_data_list.append(corner_data)

# Step 4: Add the corner data as new columns to the original DataFrame
for i in range(8):
    df[f'Corner_{i+1}'] = [corner_data[i] for corner_data in corner_data_list]

folder_path='F:/Intern_work/trash'
# Step 5: Drop any unnecessary columns (if needed) or save the DataFrame
new_file_path = os.path.join(folder_path, '3d_coord.csv')
df.to_csv(new_file_path, index=False)

print(f'Successfully saved the updated DataFrame with corner coordinates to {new_file_path}')
##########################. 












#########################  3d_to_velodyne.ipynb
import numpy as np
import pandas as pd

# Load the CSV file containing Tv2c matrix values (assuming it's named 'labels.csv')
# The CSV file should contain four columns and three rows of the Tv2c matrix


# Extract the values of Tv2c from the CSV
Tv2c = np.array([
    tv2c_df.iloc[0].values,  # First row
    tv2c_df.iloc[1].values,  # Second row
    tv2c_df.iloc[2].values,  # Third row
    [0, 0, 0, 1]  # Fourth row remains fixed
])

# Invert Tv2c to get Tc2v (transformation from camera to Velodyne coordinates)
Tc2v = np.linalg.inv(Tv2c)

# Load the CSV file named '3d_coord.csv' into a dataframe
df = pd.read_csv('3d_coord.csv')

# Function to transform a 3D point from camera to Velodyne coordinates
def transform_to_velodyne(point, Tc2v):
    point_homogeneous = np.append(point, 1)
    point_velodyne = np.dot(Tc2v, point_homogeneous)
    return point_velodyne[:3]

# Check if 'Corner_1' to 'Corner_8' columns exist in the dataframe
if all([f'Corner_{i}' in df.columns for i in range(1, 9)]):
    for i in range(1, 9):
        df[f'Corner_{i}_velodyne'] = df[f'Corner_{i}'].apply(lambda point: transform_to_velodyne(np.array(eval(point)), Tc2v))
else:
    print("Required corner columns (Corner_1 to Corner_8) are missing from the dataset.")

df.drop([f'Corner_{i}' for i in range(1, 9)], axis=1, inplace=True)

# Save the updated DataFrame to a new CSV file
new_file_path = 'velodyne_coord.csv'
df.to_csv(new_file_path, index=False)
###############################




############################### velodyne_to_frontview.ipynb
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the CSV file
file_path = 'F:/Intern_work/trash/velodyne_coord.csv'  # Update with your actual file path
df = pd.read_csv(file_path)

# Horizontal and vertical angular resolutions
Resh = 0.17  # Horizontal angular resolution in degrees
Resv = 0.4   # Vertical angular resolution in degrees

# Function to calculate Xfront and Yfront from (x, y, z)
def calculate_front_view(coord):
    # Convert string representation of the array to a numpy array
    coord = np.fromstring(coord.strip('[]'), sep=' ')
    x, y, z = coord
    d = np.sqrt(x**2 + y**2)  # Distance from (0, 0, 0) in the XY plane
    Xfront = np.arctan2(y, x) * (180 / (np.pi * Resh))  # Horizontal angle
    Yfront = np.arctan2(z, d) * (180 / (np.pi * Resv))  # Vertical angle
    return f"({Xfront}, {Yfront})"  # Return the combined (Xfront, Yfront) as a string

# Loop through each Corner_velodyne column and replace (x, y, z) with (Xfront, Yfront)
for i in range(1, 9):
    df[f'Corner_{i}_velodyne'] = df[f'Corner_{i}_velodyne'].apply(lambda coord: calculate_front_view(coord))

# Save the updated DataFrame to a new CSV file
new_file_path = 'front_view_coord.csv'  # Update with your desired new file path
df.to_csv(new_file_path, index=False)

# Display the first few rows of the updated DataFrame to verify
df.head()

# Save the updated DataFrame to a new CSV file
new_file_path = 'front_view.csv'  # Update with your desired new file path
df.to_csv(new_file_path, index=False)
#####################################





#####################################  plotting lidar data and bounding box
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Define the output folders for saving files
point_cloud_output_folder = 'F:/Intern_work/pcd_front_view/'  # Folder to store front view points cloud data
bounding_box_output_folder = 'F:/Intern_work/bb_front_view/'  # Folder to store front view bounding box data

# Ensure that the output folders exist; if not, create them
if not os.path.exists(point_cloud_output_folder):
    os.makedirs(point_cloud_output_folder)

if not os.path.exists(bounding_box_output_folder):
    os.makedirs(bounding_box_output_folder)


# Extract columns (assuming they are named 'x', 'y', 'z', and 'intensity')
x = ptCloud_df['x'].values
y = ptCloud_df['y'].values
z = ptCloud_df['z'].values
reflectivity = ptCloud_df['intensity'].values  # Use the appropriate column for intensity

# Distance from the sensor in the XY plane
d = np.sqrt(x**2 + y**2)

# Horizontal and vertical angular resolutions
Resh = 0.17  # Horizontal angular resolution in degrees
Resv = 0.4   # Vertical angular resolution in degrees

# Calculate the horizontal and vertical angles for the point cloud
Xfront = np.arctan2(y, x) * (180 / (np.pi * Resh))  # Horizontal angle
Yfront = np.arctan2(z, d) * (180 / (np.pi * Resv))  # Vertical angle

d = f'F:/Intern_work/pcd_front_view/{i:06}.csv'
e = f'F:/Intern_work/bb_front_view/{i:06}.csv'



# Save the point cloud's Xfront and Yfront to a CSV file in the 'abc' folder
pcd_output_file = os.path.join(point_cloud_output_folder, d)
point_cloud_df = pd.DataFrame({
    'Xfront': Xfront,
    'Yfront': Yfront
})
point_cloud_df.to_csv(pcd_output_file, index=False)

print(f'Successfully saved point cloud Xfront and Yfront data to {pcd_output_file}')











# Load the output CSV file with precomputed front view coordinates (3D bounding box)
labels_df = pd.read_csv('F:/Intern_work/trash/front_view_coord.csv')

# Prepare to collect front view coordinates from bounding box corners
front_view_coords = []

# Extract front view coordinates (Xfront, Yfront) from the CSV file for each of the 8 corners
for _, row in labels_df.iterrows():
    for i in range(1, 9):  # There are 8 corner coordinates (Corner_1_velodyne to Corner_8_velodyne)
        # Read (Xfront, Yfront) from the columns directly
        coord = row[f'Corner_{i}_velodyne'].strip('()').split(',')
        Xfront_box, Yfront_box = map(float, coord)
        front_view_coords.append((Xfront_box, Yfront_box))

# Convert front view coordinates into a NumPy array for saving
front_view_coords = np.array(front_view_coords)

# Save bounding box Xfront and Yfront data to a CSV file in the 'def' folder
bb_output_file = os.path.join(bounding_box_output_folder, e)
bounding_box_df = pd.DataFrame({
    'Xfront': front_view_coords[:, 0],
    'Yfront': front_view_coords[:, 1]
})
bounding_box_df.to_csv(bb_output_file, index=False)

print(f'Successfully saved bounding box Xfront and Yfront data to {e}')

# Optional: Plot the data along with bounding box corners (if needed)
# Create an interactive scatter plot using Plotly with point cloud and bounding boxes
fig = go.Figure()





# Add point cloud data (from 000000.csv)
fig.add_trace(go.Scattergl(
    x=Xfront,
    y=Yfront,
    mode='markers',
    marker=dict(
        color=reflectivity,
        colorscale=[[0, 'grey'], [1, 'white']],  # Custom colorscale from black (0 intensity) to white (max intensity)
        cmin=0,  # Set minimum color scale value
        cmax=max(reflectivity),  # Set maximum color scale value
        colorbar=dict(title="Intensity"),
        size=5,
        symbol='square'
    ),
    name='Point Cloud'
))





# Update layout for the plot (if not already included)
fig.update_layout(
    title='2D Front View of Point Cloud with Bounding Boxes and CSV Coordinates',
    xaxis_title='Xfront (Horizontal Angle)',
    yaxis_title='Yfront (Vertical Angle)',
    xaxis_scaleanchor="y",  # Anchor x-axis to y-axis
    yaxis_scaleratio=1,  # Ensure the same scaling ratio
    showlegend=True
)


fig.add_trace(go.Scatter(
    x=front_view_coords[:, 0],  # Xfront (Horizontal Angle)
    y=front_view_coords[:, 1],  # Yfront (Vertical Angle)
    mode='markers',
    marker=dict(
        color='red',
        size=6,
        symbol='x'
    ),
    name='Bounding Box Corners',
    # Ensure this trace is layered above others
    
))

fig.update_layout(
    plot_bgcolor='black',    # Background color of the plot area
    paper_bgcolor='black',   # Background color of the entire figure
    font=dict(color='white'), # Make text elements white for contrast
    xaxis=dict(
        showgrid=False,      # Remove gridlines on the x-axis
        zeroline=False,      # Remove the x-axis line at y=0
        showline=False       # Remove the outer x-axis line
    ),
    yaxis=dict(
        showgrid=False,      # Remove gridlines on the y-axis
        zeroline=False,      # Remove the y-axis line at x=0
        showline=False       # Remove the outer y-axis line
    )
)

# Show the updated plot
fig.show()







import os

# # Function to delete all CSV files from a given folder
# def delete_csv_files(folder_path):
#     # Get a list of all files in the folder
#     files_in_folder = os.listdir(folder_path)

#     # Loop through each file in the folder
#     for file_name in files_in_folder:
#         # Check if the file is a CSV file
#         if file_name.endswith('.csv'):
#             # Construct full file path
#             file_path = os.path.join(folder_path, file_name)

#             # Delete the file
#             os.remove(file_path)
#             print(f'Deleted: {file_path}')

# # Example usage
# folder_path = 'F:/Intern_work/trash/'  # Update with the path to your folder
# delete_csv_files(folder_path). analyse above code and write only "OK" 